![Cost%20Effective%20Active%20Learning%20for%20Deep%20Image%20Classification.png](attachment:Cost%20Effective%20Active%20Learning%20for%20Deep%20Image%20Classification.png)

In [106]:
import tensorflow as tf
from keras.datasets import cifar10
from keras.applications import mobilenet, nasnet
from keras import backend as K
from keras.utils.np_utils import to_categorical
from keras.preprocessing.image import ImageDataGenerator
from keras.models import load_model
from keras.callbacks import ModelCheckpoint, TensorBoard, ReduceLROnPlateau, EarlyStopping, ProgbarLogger

from keras.layers import Input, Dense, GlobalAveragePooling2D, Reshape, Conv2D

from keras.models import Model
import numpy as np
# from keras.utils import plot_model
# from IPython.display import SVG
# from keras.utils.vis_utils import model_to_dot


num_classes = 10

(x_train, y_train), (x_test, y_test) = cifar10.load_data()

y_train = to_categorical(y_train, num_classes)
y_test = to_categorical(y_test, num_classes)

batch_size = 32
initial_train_perc = 0.2
initial_train_size = int(x_train.shape[0] * initial_train_perc)

datagen = ImageDataGenerator()

x_train_initial, y_train_initial = iter(datagen.flow(x_train, y_train, batch_size=initial_train_size, shuffle=True)).next()

print(x_train_initial.shape)
print(y_train_initial.shape)

print(x_train.shape)
print(y_train.shape)

input_shape = x_train[-1,].shape
# input_tensor = Input(shape=input_shape)

def _nasnet(num_classes, input_shape=(32,32,3), pretrained=True, freezed=True):
    input_tensor = Input(shape=input_shape)

    weights = 'imagenet' if pretrained else None
    base_model = nasnet.NASNetMobile(input_tensor=input_tensor, weights=weights, include_top=False)
    x = base_model.output
    x = GlobalAveragePooling2D()(x)
    predictions = Dense(num_classes, activation='softmax', name='predictions')(x)
    if freezed:
        for layer in base_model.layers:
            layer.trainable = False
    model = Model(inputs=base_model.input, outputs=predictions)
    return model

def _mobilenet(num_classes, input_shape=(32,32,3), pretrained=True, freezed=True):
    input_tensor = Input(shape=input_shape)

    weights = 'imagenet' if pretrained else None
    base_model = mobilenet.MobileNet(input_tensor=input_tensor, weights=weights, include_top=False)
    x = base_model.output
    x = GlobalAveragePooling2D()(x)
    x = Reshape(shape, name='reshape_1')(x)
    x = Dropout(dropout, name='dropout')(x)
    x = Conv2D(classes, (1, 1), padding='same', name='conv_preds')(x)
    x = Activation('softmax', name='act_softmax')(x)
    predictions = Reshape((num_classes,), name='reshape_2')(x)
    if freezed:
        for layer in base_model.layers:
            layer.trainable = False
    model = Model(inputs=base_model.input, outputs=predictions)
    return model
        
model = _nasnet(num_classes, input_shape)
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['acc'])

# SVG(model_to_dot(model).create(prog='dot', format='svg'))
# model = load_model('ceal_initial_nasnet.hdf5')
# model.summary()


# checkpointer = ModelCheckpoint(filepath='ceal_initial_nasnet.hdf5', verbose=1, save_best_only=True)
# reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=5, min_lr=0.001)
# tensorboard = TensorBoard()
# progbar = ProgbarLogger()
# earlystop = EarlyStopping(patience=5)

hist = model.fit(x_train_initial, y_train_initial, validation_data=(x_test, y_test), epochs=5)
#                  callbacks=[checkpointer, reduce_lr, tensorboard, progbar, earlystop])


(10000, 32, 32, 3)
(10000, 10)
(50000, 32, 32, 3)
(50000, 10)
Train on 10000 samples, validate on 10000 samples
Epoch 1/5
10000/10000 [==============================] - 33s 3ms/step - loss: 2.6408 - acc: 0.2196 - val_loss: 14.6165 - val_acc: 0.0872
Epoch 2/5
10000/10000 [==============================] - 14s 1ms/step - loss: 2.1483 - acc: 0.2929 - val_loss: 14.7511 - val_acc: 0.0769
Epoch 3/5
10000/10000 [==============================] - 14s 1ms/step - loss: 2.0571 - acc: 0.3127 - val_loss: 14.6288 - val_acc: 0.0798
Epoch 4/5
10000/10000 [==============================] - 14s 1ms/step - loss: 2.0258 - acc: 0.3158 - val_loss: 14.7057 - val_acc: 0.0831
Epoch 5/5
10000/10000 [==============================] - 14s 1ms/step - loss: 2.0217 - acc: 0.3169 - val_loss: 14.6324 - val_acc: 0.0832


In [2]:
"""
Adapted from keras example cifar10_cnn.py and github.com/raghakot/keras-resnet
Train ResNet-18 on the CIFAR10 small images dataset.

GPU run command with Theano backend (with TensorFlow, the GPU is automatically used):
    THEANO_FLAGS=mode=FAST_RUN,device=gpu,floatX=float32 python cifar10.py
"""
from __future__ import print_function
from keras.datasets import cifar10
from keras.preprocessing.image import ImageDataGenerator
from keras.utils import np_utils
from keras.callbacks import ModelCheckpoint
from keras.callbacks import ReduceLROnPlateau
from keras.callbacks import CSVLogger
from keras.callbacks import EarlyStopping
from keras_contrib.applications.resnet import ResNet18

import numpy as np


weights_file = 'ResNet18v2-CIFAR-10.h5'
lr_reducer = ReduceLROnPlateau(factor=np.sqrt(0.1), cooldown=0, patience=5, min_lr=0.5e-6)
early_stopper = EarlyStopping(min_delta=0.001, patience=10)
csv_logger = CSVLogger('ResNet18v2-CIFAR-10.csv')
model_checkpoint = ModelCheckpoint(weights_file, monitor='val_acc', save_best_only=True,
                                   save_weights_only=True, mode='auto')

batch_size = 32
nb_classes = 10
nb_epoch = 50

# input image dimensions
img_rows, img_cols = 32, 32
# The CIFAR10 images are RGB.
img_channels = 3

# The data, shuffled and split between train and test sets:
(X_train, y_train), (X_test, y_test) = cifar10.load_data()

# Convert class vectors to binary class matrices.
Y_train = np_utils.to_categorical(y_train, nb_classes)
Y_test = np_utils.to_categorical(y_test, nb_classes)

X_train = X_train.astype('float32')
X_test = X_test.astype('float32')

# subtract mean and normalize
mean_image = np.mean(X_train, axis=0)
X_train -= mean_image
X_test -= mean_image
X_train /= 128.
X_test /= 128.

model = ResNet18((img_rows, img_cols, img_channels), nb_classes)
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])
model.fit(X_train, Y_train,
          batch_size=batch_size,
          nb_epoch=nb_epoch,
          validation_data=(X_test, Y_test),
          shuffle=True,
          callbacks=[lr_reducer, early_stopper, csv_logger, model_checkpoint])

scores = model.evaluate(X_test, Y_test, batch_size=batch_size)
print('Test loss : ', scores[0])
print('Test accuracy : ', scores[1])

reshaping via a convolution...
reshaping via a convolution...
reshaping via a convolution...
reshaping via a convolution...
Not using data augmentation.


/home/daniel/.local/lib/python3.6/site-packages/ipykernel_launcher.py:67: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.


Train on 50000 samples, validate on 10000 samples
Epoch 1/50
50000/50000 [==============================] - 102s 2ms/step - loss: 1.7161 - acc: 0.5075 - val_loss: 1.4815 - val_acc: 0.5608
Epoch 2/50
50000/50000 [==============================] - 98s 2ms/step - loss: 1.3180 - acc: 0.6297 - val_loss: 1.2774 - val_acc: 0.6414
Epoch 3/50
50000/50000 [==============================] - 98s 2ms/step - loss: 1.1413 - acc: 0.6853 - val_loss: 1.1574 - val_acc: 0.6750
Epoch 4/50
50000/50000 [==============================] - 98s 2ms/step - loss: 1.0409 - acc: 0.7203 - val_loss: 1.0843 - val_acc: 0.7039
Epoch 5/50
50000/50000 [==============================] - 98s 2ms/step - loss: 0.9568 - acc: 0.7485 - val_loss: 1.0565 - val_acc: 0.7138
Epoch 6/50
50000/50000 [==============================] - 98s 2ms/step - loss: 0.8839 - acc: 0.7750 - val_loss: 1.0374 - val_acc: 0.7172
Epoch 7/50
50000/50000 [==============================] - 98s 2ms/step - loss: 0.8296 - acc: 0.7935 - val_loss: 1.0089 - val_ac

In [4]:
from sklearn.metrics import classification_report, confusion_matrix
import scipy as sc
import heapq


(x_train, y_train), (x_test, y_test) = cifar10.load_data()

def least_confidence(y_pred_prob, y_true):
    
    origin_index = np.arange(0,len(y_pred_prob))
    max_prob = np.max(y_pred_prob, axis=1)
    max_prob_index = np.argmax(y_pred_prob, axis=1)
    
    lci = np.column_stack((origin_index,
                            max_prob,
                            max_prob_index, 
                            y_true))
    
    lci = lci[lci[:,1].argsort()]
    return lci, lci[:,0].astype(int)

def margin_sampling(y_pred_prob, y_true):
    
    origin_index = np.arange(0,len(y_pred_prob))
    max_prob = np.max(y_pred_prob, axis=1)
    max_prob_index = np.argmax(y_pred_prob, axis=1)
    
    for row in y_pred_prob:
#         a = heapq.nlargest(2, range(len(row)), row.take)
        a = row.argsort()[-2:][::-1]
        b = np.take(row, a)
#   
    return np.sort(np.amax(y_pred_prob, axis=1))

def entropy(y_pred_prob, y_true):
#     entropy = sc.stats.entropy(y_pred_prob, base=2, axis=1)
#     entropy = np.nan_to_num(entropy)
    origin_index = np.arange(0,len(y_pred_prob))
    max_prob = np.max(y_pred_prob, axis=1)
    max_prob_index = np.argmax(y_pred_prob, axis=1)
    entropy = -np.nansum(np.multiply(y_pred_prob, np.log(y_pred_prob)), axis=1)
    eni = np.column_stack((origin_index,
                            entropy,
                            max_prob,
                            max_prob_index, 
                            y_true))
                           
    eni = eni[(-eni[:,1]).argsort()]
    return eni, eni[:,0].astype(int)

def high_confidence(y_pred_prob, y_true, delta):
    eni, eni_idx = entropy(y_pred_prob, y_true)
    hcs = eni[eni[:,1] < delta]
    return hcs, hcs[:,0].astype(int)


##### CEAL parameters #####

#maximum iteration numbers
T=50
#fine-tuning interval
t=1
#threshold decay rate
dr= 0.00033
#high confidence samples selection threshold
delta=0.005
#uncertain samples selection size
K=1000

#unlabeled samples
DU = None
#initially labeled samples
DL = None
#high confidence samples
DH = None

dataset_size = 1000
num_classes = 10

np.random.seed(1)

x = x_train#[0:dataset_size]
y = y_train#[0:dataset_size]

# y_pred_prob = np.random.rand((y.shape[0]), num_classes)
# y_pred_prob = y_pred_prob / y_pred_prob.sum(axis=1, keepdims=True)
# y_pred_prob = np.around(y_pred_prob, 3)
# print(y_pred_prob)
# print(np.sum(y_pred_prob))

for i in range(T):  
    y_pred_prob = model.predict(x, verbose=0)
#     y_pred_prob = np.around(y_pred_prob, 3)

    isa, isa_idx = least_confidence(y_pred_prob, y)
    # isa, isa_idx = margin_sampling(y_pred_prod, y)
    # isa, isa_idx = entropy(y_pred_prob, y)
    hcs, hcs_idx = high_confidence(y_pred_prob, y, delta)
    
    idx_concat = np.concatenate((isa_idx, hcs_idx),0)
    idx = np.unique(idx_concat, return_index=True)[1]
    idx = np.array([idx_concat[i] for i in sorted(idx)])
    
    step = i*K
    DH = np.take(x, idx[step+K:], axis=0), np.take(y, idx[step+K:], axis=0)
    DL = np.take(x, idx[step:step+K], axis=0), np.take(y, idx[step:step+K], axis=0)
    x = np.delete(x, idx[step:step+K], axis=0)
    y = np.delete(y, idx[step:step+K], axis=0)
    print(x.shape)
    
#     x_uncertain = np.take(x, isa_idx[step:step+K], axis=0)
#     y_uncertain = np.take(y, isa_idx[step:step+K], axis=0)
    
#     if DL is None:
#         DL = x_uncertain, y_uncertain
#     else:
#         x_l, y_l = DL
#         x_l = np.append(x_l, x_uncertain, axis=0)
#         y_l = np.append(y_l, y_uncertain, axis=0)
#         DL = x_l, y_l
        
    al_x, al_y = np.append(DL[0], DH[0], axis=0), np.append(DL[1], DH[1], axis=0)
#     print(DL[0].shape)
#     print(DL[1].shape)
#     print(DH[0].shape)
#     print(DH[1].shape)
#     print(x.shape)
#     print(y.shape)
#     print(al_x.shape)
#     print(al_y.shape)
#     print(x_train_initial.shape)
#     print(y_train_initial.shape)
    if i % t == 0:
        model.fit(al_x, al_y, epochs=5, verbose=0)
        delta -= (dr * (i//t))
    evaluate = model.evaluate(x_test, y_test, verbose=1)
    print(evaluate)


# print(classification_report(np.argmax(y, axis=1), y_pred_idx))
# print(confusion_matrix(np.argmax(y, axis=1), y_pred_idx))

/home/daniel/.local/lib/python3.6/site-packages/ipykernel_launcher.py:41: RuntimeWarning: divide by zero encountered in log
/home/daniel/.local/lib/python3.6/site-packages/ipykernel_launcher.py:41: RuntimeWarning: invalid value encountered in multiply


(49000, 32, 32, 3)


ValueError: Error when checking target: expected dense_2 to have shape (10,) but got array with shape (1,)